### IMPORTS

In [4]:
import subprocess
import os
import json
import re

### CONSTANTS

In [5]:
BASE_DIR = "E:\Workspace\Work"

### HELPER FUNCTIONS

In [6]:
# remove all ANSI escape sequences from a string
def removeEscapeSequences(s):
    return re.sub(r'\x1b[^m]*m', '', s)

### UPDATE JSON FILES

In [7]:
# Update Projects
with open("data/projs.json", "r") as f:
    projects = json.load(f)

# Update Repositories
with open("data/repos.json", "r") as f:
    repositories = json.load(f)

### CLONE GLAB PROJECTS INTO FOLDER STRUCTURE

In [11]:
for project in projects:
    # create project folder with full path
    project_folder = os.path.join(BASE_DIR, project["path_with_namespace"])

    # check if the project is already cloned
    if os.path.exists(os.path.join(BASE_DIR, project["path_with_namespace"])):
        print(f"Project {project['name']} is already cloned")
        continue
    # run glab clone command
    subprocess.run(
        f"glab repo clone {project['path_with_namespace']} {project_folder}",
        shell=True,
        check=True,
    )
    print(f"Cloned project {project['name']} to {project_folder}")

Project RealEstateScrapeBot is already cloned
Project OneTrueGit is already cloned
Project MAT120 is already cloned
Project CSE421 is already cloned
Project CSE250 is already cloned
Project CSE221 is already cloned
Project CSE220 is already cloned
Project CSE111 is already cloned
Project CSE110 is already cloned
Project The Library is already cloned
Project jupyter-ipyreact is already cloned
Project Data Structure is already cloned
Project LAB-temp is already cloned
Project BracU-App is already cloned
Project BookStore is already cloned
Project CarRepairDashboardApp is already cloned


### GOTO EACH PROJECT AND CREATE A GITHUB REPO

In [33]:
allPathWithNamespace = map(lambda x: x['path_with_namespace'], projects)


allRepoNames = map(
    lambda x: x['full_name'].replace("SafwanAmin-BracU/", ""),
    filter(
        lambda x: not x['archived'],
        repositories
    )
)
allRepoNames = set(allRepoNames)
print(allRepoNames)

set()


In [34]:
def gHubNamingFormat(q: str): return q.replace("/", "__")
def gLabNamingFormat(q: str): return q.replace("__", "/")


def syncWithGitHub(proj: dict):
    print(f"Syncing project {proj['name']} with github")

    # get local path of the project
    nameSpace = proj['path_with_namespace']
    # get full path of the project
    repoPath = os.path.join(BASE_DIR, nameSpace)
    # generate the github repo name
    githubRepoName = gHubNamingFormat(nameSpace)

    # CREATE THE REPO IN GITHUB: (IF NOT EXISTS)

    os.chdir(repoPath)
    # check if the project is already exists in github or not
    if githubRepoName in allRepoNames:
        print(f"Project {proj['name']} is already in github")
    else:
        # if not, create the repo in github
        print(f"Creating github repo for project {proj['name']}")
        subprocess.run(
            f"gh repo create {githubRepoName} --{project['visibility']}",
            shell=True,
            check=True,
        )

    print(f"Github repo for project {proj['name']} is created")
    # add the github repo as remote to the project (This will add the github repo as a remote to the local project)
    print(f"Adding github repo as remote to project {proj['name']}")
    repoUrl = f"https://github.com/SafwanAmin-BracU/{githubRepoName}.git"
    subprocess.run(
        f"git remote set-url --add --push origin {repoUrl}",
        shell=True,
        check=True,
    )

    # push the project to the github repo (This will update the github repo with the local project)
    print(f"Pushing project {proj['name']} to github")
    subprocess.run(
        "git push --all origin",
        shell=True,
        check=True,
    )
    print(f"Project {proj['name']} is synced with github\n\n")


for project in projects:
    syncWithGitHub(project)

Syncing project RealEstateScrapeBot with github
Creating github repo for project RealEstateScrapeBot
Github repo for project RealEstateScrapeBot is created
Adding github repo as remote to project RealEstateScrapeBot
Pushing project RealEstateScrapeBot to github


CalledProcessError: Command 'git push origin master' returned non-zero exit status 1.